In [1]:
###VGGNET in Pytorch

In [2]:
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [3]:
class VGGnet(nn.Module):
    def __init__(self,modle,in_channel=3,num_class=10): 
        super(VGGnet,self).__init__()
        
        self.in_channels=in_channels
        
        #모델별 구조 딕셔너리로 define->convolution과 maxpooling 구조 반복하면서 이미지 사이즈는 줄여주고 채널수는 늘리면서 층을 깊게 쌓는 구조
        self.VGG_types = {
            'VGG11': [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M'],
            'VGG13': [64,64,'M',128,128,'M',256,256,'M',512,512,'M',512,512,'M'],
            'VGG16': [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M'],
            'VGG19': [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M']
        }
        
        #create conv layer
        self.conv_layer=self.create_conv_laters(self.VGG_types[model]) #model에 VGG11,13,16,19등 사용할 모델명을 삽입해줌. create_conv_laters는 따로 define한 함수
        
        #fully connected layer define
        self.fcs=nn.Sequential(
            nn.Linear(512*7*7,2048),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024,num_class),
        )
        
        def forward(self,x):
            x=self.conv_layer(x)
            x=x.view(-1,512*7*7)
            x=self.fcs(x)
            return x
        
        #define a function to create conv layer
        def create_conv_laters(self,architecture): #filter 3x3 고정, stride=1x1 
            layers = [] #빈리스트 생성
            in_channels=self.in_channels #3
            
            for x in architecture: 
                if type(x) == int:
                    out_channels = x #첫번째 블록 out channle 입력, self.VGG_types 리스트에서 하나씩->architecture=VGG_types list
                    #nn. Conv2d (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1,
                    layers += [nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                               nn.BatchNorm2d(x),
                               nn.ReLU()]
                    in_channels=x #한번 끝나면 out_channels->in_channels로
                elif x == 'M':
                    layers += [nn.MaxPool2d(kernel_size(2,2), stride=(2,2))]
                    
            return nn.Sequential(*layers)
        

In [4]:
###VGG16 Model Define VGG 11,13,16,19

In [5]:
class VGG16Model(nn.Module):
    def __init__(self,train_dataset,val_dataset,num_classes,epoch,learning_rate,batch_size,loader_num_workers):
        self.device=torch.device("cuda" if torch.cuda.is_availabel() else 'cpu')
        self.train_dataset=train_dataset
        self.val_dataset=val_dataset
        self.num_classes=num_classes
        self.epoch=epoch
        self.learning_rate=learning_rate
        self.batch_size=batch_size
        self.loader_num_workeres=loader_num_workers
        self.patience_i= int(min(np.ceil(epoch*0.1), 10))  #이후에 lr scheduler parameter
        
    def VGG16Train(self):
        model = VGGnet('VGG16', in_channels=3, num_classes=self.num_classes).to(self.device) # model gpu로 보내줌
        
        criterion=nn.CrossEntropyLoss()
        optimizer=optim.Adam(model.parameters(),lr=self.learning_rate,weight_decay=1e-4)
        lr_scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=self.patience_i,verbose=True) #val loss 정체되면 learning rate 조정
        
        train_loader=DataLoader(self.train_dataset,batch_size=self.batch_size, shuffle=True, num_workers=self.loader_num_workers) #train set
        val_loader=DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False, num_workers=self.loader_num_workers) #val set
        dataloaders={'train': train_loader, 'val':val_loader}
        
        since=time.time()
        train_acc_history=[]
        train_loss_history=[]
        val_acc_history=[]
        val_loss_history=[]
        best_model_wts=copy.deepcopy(model.state_dict())
        best_acc=0.0
        use_auxiliary=False
        
        for i in range(self.epoch):
            epoch_time=time.time()
            print('Epoch {}/{}'.format(i+1, self.epoch))
            print('-'*10)
            
            for phase in ['train','val']:
                if phase=='train':
                    model.train()
                else:
                    model.eval()
                    
                running_loss=0.0
                running_corrects=0
                
            for inputs,labels in dataloaders[phase]:
                
                inputs=inputs.to(self.device)
                
                labels=labels.to(self.device)
                
                optimizer.zero_grad() #한번 학습이 완료되면, gradient를 0으로 만들어주어야함, 초기화하지 않을시 의도한 방향과 다른방향으로 학습할 가능성
                
                with torch.set_grad_enabled(phase=='train'):
                    if phase=='train':
                        if use_auxiliary:
                            outputs, aux1, aux2 = model(inputs)
                            loss = criterion(outputs,labels) + 0.3 *criterion(aux1,labels) +0.3*criterion(aux2,labels)
                        else:
                            outputs=model(inputs)
                            loss=criterion(outputs,labels)
                            
                        _,preds = torch.max(outputs,1)
                        loss.backward()
                        optimizer.step()
                        
                    if phase == 'val':
                        outputs = models(inputs)
                        loss=criterion(outputs,labels)
                        _,preds=torch.max(outputs,1)
                        
                #통계값 출력 Part
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset) #loss평균 계산 전체데이터
            
            if phase == 'val':
                lr_scheduler.step(epochh_loss)
                
            epoch_acc = running_corrects.double()/len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            e_time = time.time() - epoch_time
            
            #deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                    val_acc_history.append(float(epoch_acc))
                    val_loss_history.append(float(epoch_loss))
            if phase == 'train':
                    train_acc_history.append(float(epoch_acc))
                    train_loss_history.append(float(epoch_loss))
                    
        print('Epoch Time : {:.0f}m {:.0f}s'.format(e_time//60, e_time%60))
        print()
        
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
        
        return model, self.epoch, train_loss_history, train_acc_history, val_loss_history, val_acc_history
    
                
            